In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

In [2]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = torch.nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = torch.nn.Linear(in_features=120, out_features=60)
        self.out = torch.nn.Linear(in_features=60, out_features=10)
        
    def forward(self,t):
        # 1. input layer
        t = t
        # 2. hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2, stride=2)
        
        # 3. hidden conv layer 2
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2, stride=2)
        
        # 4. linear layer1
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # 5. linear layer 2
        t = self.fc2(t)
        t = F.relu(t)
        
        # 6. output layer
        t = self.out(t)
#         t = F.softmax()
        
        return t

In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [6]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)

# Starting out with TensorBoard

In [7]:
tb = SummaryWriter()

network = Network()
images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images',grid)
tb.add_graph(network,images)
tb.close()

The training loop review

In [9]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100,shuffle=True)
optimizer = torch.optim.Adam(network.parameters(),lr=0.01)

tb = SummaryWriter()
images,labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

for epoch in range(5):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader:
        images,labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds,labels)   #计算loss

        optimizer.zero_grad()
        loss.backward()    #计算梯度
        optimizer.step()   #更新权重

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    
    tb.add_scalar('Loss', total_loss,epoch)
    tb.add_scalar('Number Correct', total_correct,epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias,epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight,epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad,epoch)

    print('epochs:',epoch,'total_loss:',total_loss,'total_correct:',total_correct)
    
tb.close()

epochs: 0 total_loss: 351.68638774752617 total_correct: 46609
epochs: 1 total_loss: 232.09728655219078 total_correct: 51361
epochs: 2 total_loss: 212.47749714553356 total_correct: 52158
epochs: 3 total_loss: 203.8905821442604 total_correct: 52398
epochs: 4 total_loss: 197.69732302427292 total_correct: 52633
